# Analysis of hover sequence

## Import statements and notebook settings

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

import seaborn as sns

from tools.data_loader import moving_average

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import glob
import os

In [ ]:
# declare ALL list objects necessary

v_induced = 14.345219306215128 # induced velocity for firefly vehicle
markersize= 15 # global markersize setting for scatter plots

# set colors

lowCol = '#FF7518' # pumpkin orange
upCol = '#18A2FF' # complementary color

upMeanCol = 'blue'
lowMeanCol = 'black'

armCol = upCol
armMeanCol = 'red'

## Helper functions (might be moved to external module)

In [ ]:
def normalization(data_vector, method='zscore'):
    """ Function to perform minmax or zscore normalization"""
    
    if method == 'minmax':
        minimum = np.min(data_vector)
        maximum = np.max(data_vector)
    
        return (data_vector - minimum) / (maximum - minimum)
    
    if method == 'zscore':
        return (data_vector - np.mean(data_vector)) / np.std(data_vector)

In [ ]:
# get the .csv files for the given velocity limit
def get_files_for_limit(limit, flight, discard=True):
    
    v_limit = limit * 0.01 * v_induced
    data_directory = flight + f'hover_{limit}_limit'
    csv_files = glob.glob(os.path.join(data_directory, "*.csv"))
    
    if limit==10 and not csv_files:
        data_directory = flight + f'hover_{100}_limit'
        csv_files = glob.glob(os.path.join(data_directory, "*.csv"))
    
    sorted_files = []

    # sort sequences in ascending order
    for i in range(len(csv_files)):
        for e in csv_files:
            if f'sequence_{i}_' in e:
                #print(e)
                sorted_files.append(e)
                break
    
    # discard first and land sequence (vehicle on ground before takeoff and after landing)
    if discard:
        return sorted_files[1:-1]
    
    else:
        return sorted_files
    

## Load flight data

In [ ]:
# select filepath where flight data is stored

# first testflight
#flight = f'../flight_data/2022-08-01_ag_field/flight_6_first_hover_flight/'
#hover_start = 35
#hover_end = 15

# second testflight
#flight = f'../flight_data/2022-08-01_ag_field/flight_8_second_hover_flight/'
#hover_start = 30
#hover_end = 30

# third testflight
flight = f'../flight_data/2022-08-29_ag_field/flight_2_delta0_sweep/'
hover_start = 30
hover_end = 30

# Check if the limits match the files in the folder
csv_files = get_files_for_limit(1, flight, discard=True)

# csv_files for maximum velocity limit (should normally contain data of the full flight)
csv_files_full = get_files_for_limit(10, flight, discard=False)

# list to store hover sequences
sequence = pd.DataFrame()

# pandas data frame to store data for full flight
full_flight = pd.DataFrame()

# stack sequences
for file in csv_files:
    if len(pd.read_csv(file)) > 30:
        sequence = pd.concat([sequence, pd.read_csv(file)])    

        # read data for full flight from csv data
for file_full in csv_files_full:
    full_flight = pd.concat([full_flight, pd.read_csv(file_full)])

## Preprocessing

### Rename columns and calculate power values for motors, arms and vehicle

In [ ]:
# choose sequence or full flight data 
fd = full_flight 
#flight_data = sequence

# cleaning and renaming of flight_data dataframe

fd = fd.rename(columns={'nsh[0]' : 'delta0'})
fd = fd.rename(columns={'ctrl_1' : 'uIn1', 'ctrl_2' : 'uIn2', 'ctrl_3' : 'uIn3', 'ctrl_4' : 'uIn4'})

fd = fd.drop(columns=['time_boot_ms', 'status', 'nooutputs', 'ctrl_5', 'ctrl_6', 'ctrl_7', 'ctrl_8', ' nsh[1]'])

fd['t'] = ((fd['t'].values - min(fd['t'].values)) / 10e5)

# rename some columns to more intuitive names, added bias from calibration to current and voltage
# calculate motor power

for i in range(1,9):
    fd = fd.rename(columns={f'U1{i}' : f'U{i}', f'I1{i}' : f'I{i}', f'omega{i}' : f'rpm{i}', f'delta_{i}': f'uOut{i}'})
    #fd[f'U{i}'] = fd[f'U{i}'] - motorVoltageBias[i-1]
    #fd[f'I{i}'] = fd[f'I{i}'] - motorCurrentBias[i-1]
    #fd[f'pMo{i}'] = fd[f'U{i}'] * fd[f'I{i}']
    
    fd = fd.drop(columns=[f'thr_in{i}'])
    fd = fd.drop(columns=[f'thr_out{i}'])
    fd = fd.drop(columns=[f'pwm_{i}'])
    fd = fd.drop(columns=[f'output{i}'])

# delta RPM for each individual arm
#fd[f'dRpmArm1'] = fd['rpm1'] - fd['rpm6']
#fd[f'dRpmArm2'] = fd['rpm2'] - fd['rpm5']
#fd[f'dRpmArm3'] = fd['rpm3'] - fd['rpm8']
#fd[f'dRpmArm4'] = fd['rpm4'] - fd['rpm7']

# add power for each individual arm
#fd[f'pArm1'] = fd['pMo1'] + fd['pMo6']
#fd[f'pArm2'] = fd['pMo2'] + fd['pMo5']
#fd[f'pArm3'] = fd['pMo3'] + fd['pMo8']
#fd[f'pArm4'] = fd['pMo4'] + fd['pMo7']

# calculate total vehicle power
#fd['pVehicle'] = fd['pArm1'] + fd['pArm2'] + fd['pArm3'] + fd['pArm4']

### Apply calibration for ESC current and voltage

In [ ]:
# correct data with biases from motor calibration script

useCalibration = True

if useCalibration:
    motorCurrentBias = np.array([6.36, 2.10, -2.27, 0.09, 6.22, 3.01, -2.66, -1.55])
    motorVoltageBias = np.array([0.07, 0.25, 0.24, 0.24, 0.17, 0.46, -0.04, 0.14]) # bias from ground test
    additionalBias = np.array([0, 0, 0.04, 0, 0, 0, 0, -0.04]) # bias through aggregated flight data

else:
    motorCurrentBias = np.zeros(8)
    motorVoltageBias = np.zeros(8)
    additionalBias = np.zeros(8)
    
for i in range(1,9):
    fd[f'U{i}'] = fd[f'U{i}'] - motorVoltageBias[i-1]
    fd[f'I{i}'] = fd[f'I{i}'] - motorCurrentBias[i-1]

In [ ]:
# discard the first 27 and last 29 seconds of the flight to remove take-off and landing sequence
hover = np.arange(len(fd))[hover_start*30:-hover_end*30]

time = fd['t'].values

# get position and velocities
x = fd['x'].values
y = fd['y'].values
z = fd['z'].values

u = fd['u'].values
v = fd['v'].values
w = fd['w'].values

p = fd['p'].values
q = fd['q'].values
r = fd['r'].values


# get delta0 value
delta0 = fd['delta0'].values
delta0_values = np.unique(delta0)

# get RPM, voltage, current and control input
omega = [fd[f'rpm{i}'] for i in range(1,9)]
voltage = [fd[f'U{i}'] for i in range(1,9)]
current = [fd[f'I{i}'] for i in range(1,9)]
control = [fd[f'uIn{i}'] for i in range(1,5)]

# calculate power for each motor

power = [voltage[i] * current[i] for i in range(0,8)]

# calcualte power for each arm
power_pair = {'16': power[0] + power[5], '25': power[1] + power[4], '38': power[2] + power[7], '47': power[3] + power[6]}

# calculate total power and average RPM
total_power = sum(power)
avg_omega = sum(omega)/8

# calculate norm of velocity
v_norm = np.sqrt(u ** 2 + v ** 2 + w ** 2)

In [ ]:
# calculate mean power for each delta0
meanPowerDelta0 = []
meanRPM = []

for i in range(8):
    motorMeanPower = [np.mean(power[i].values[np.intersect1d(np.where(delta0==value),hover)]) for value in delta0_values]
    motorMeanRPM = [np.mean(omega[i].values[np.intersect1d(np.where(delta0==value),hover)]) for value in delta0_values]
    meanPowerDelta0.append(motorMeanPower)
    #print(motorMeanPower[0])
    meanRPM.append(motorMeanRPM)

# calculate deltaRPM
deltaRPM = {'16': omega[0] - omega[5], '25': omega[1] - omega[4], '38': omega[2] - omega[7], '47': omega[3] - omega[6]}

# TODO: create classes for motors and arms?
meanPowerArm = []
meanPowerArm.append([sum(x) for x in zip(meanPowerDelta0[1], meanPowerDelta0[4])])
meanPowerArm.append([sum(x) for x in zip(meanPowerDelta0[0], meanPowerDelta0[5])])
meanPowerArm.append([sum(x) for x in zip(meanPowerDelta0[2], meanPowerDelta0[7])])
meanPowerArm.append([sum(x) for x in zip(meanPowerDelta0[3], meanPowerDelta0[6])])

vehiclePower = [sum(x) for x in zip(*meanPowerArm)]

deltaRpmArm = []
deltaRpmArm.append([(x - y) for x,y in zip(meanRPM[1], meanRPM[4])])
deltaRpmArm.append([(x - y) for x,y in zip(meanRPM[0], meanRPM[5])])
deltaRpmArm.append([(x - y) for x,y in zip(meanRPM[2], meanRPM[7])])
deltaRpmArm.append([(x - y) for x,y in zip(meanRPM[3], meanRPM[6])])

In [ ]:
# calculate median power for each delta0

medianPowerDelta0 = []
medianRPM = []

for i in range(8):
    motorMedianPower = [np.median(power[i].values[np.intersect1d(np.where(delta0==value),hover)]) for value in delta0_values]
    motorMedianRPM = [np.median(omega[i].values[np.intersect1d(np.where(delta0==value),hover)]) for value in delta0_values]
    medianPowerDelta0.append(motorMedianPower)
    #print(motorMedianPower[0])
    medianRPM.append(motorMedianRPM)

# TODO: create classes for motors and arms?
medianPowerArm = []
medianPowerArm.append([sum(x) for x in zip(medianPowerDelta0[1], medianPowerDelta0[4])])
medianPowerArm.append([sum(x) for x in zip(medianPowerDelta0[0], medianPowerDelta0[5])])
medianPowerArm.append([sum(x) for x in zip(medianPowerDelta0[2], medianPowerDelta0[7])])
medianPowerArm.append([sum(x) for x in zip(medianPowerDelta0[3], medianPowerDelta0[6])])

medianVehiclePower = [sum(x) for x in zip(*medianPowerArm)]

deltaRpmArm = []
deltaRpmArm.append([(x - y) for x,y in zip(medianRPM[1], medianRPM[4])])
deltaRpmArm.append([(x - y) for x,y in zip(medianRPM[0], medianRPM[5])])
deltaRpmArm.append([(x - y) for x,y in zip(medianRPM[2], medianRPM[7])])
deltaRpmArm.append([(x - y) for x,y in zip(medianRPM[3], medianRPM[6])])

In [ ]:
# Calculate motor cmds

# build control inout vector
ctrl_input = np.array([control[0].values, control[1].values, control[2].values, control[3].values,
                      delta0, delta0,
                      delta0, delta0])

B_plus = np.array([[-1.4142,  1.4142,  2.0000, 2.0000, 0.4981, 0.0019, -0.0019, 0.0019],
              [ 1.4142,  1.4142, -2.0000, 2.0000, 0.0019, 0.4981,  0.0019,-0.0019],
              [ 1.4142, -1.4142,  2.0000, 2.0000,-0.0019, 0.0019,  0.4981, 0.0019],
              [-1.4142, -1.4142, -2.0000, 2.0000, 0.0019,-0.0019,  0.0019, 0.4981],
              [ 1.4142,  1.4142,  2.0000, 2.0000,-0.0019,-0.4981, -0.0019, 0.0019],
              [-1.4142,  1.4142, -2.0000, 2.0000,-0.4981,-0.0019,  0.0019,-0.0019],
              [-1.4142, -1.4142,  2.0000, 2.0000,-0.0019, 0.0019, -0.0019,-0.4981],
              [ 1.4142, -1.4142, -2.0000, 2.0000, 0.0019,-0.0019, -0.4981,-0.0019]])

delta_cmd = (B_plus @ ctrl_input) -1

for i in range(1,9):
    fd[f'uOut{i}'] = delta_cmd[i-1,:]

### Correct current signal for motor 3
For unknown reasons the current sensor of motor 3 is not capable of detecting values below 2.66 A. This can lead to a bias in the power signal for motor 3. For this reason, we perform a linear regression for current and RPM signal, as the RPM signal is considered more reliable. All values below 2.66 A are replaced with the predictions for the RPM value  

In [ ]:
# Perform regression RPM vs. current

rpm_range = np.linspace(0, 3000, 100).reshape(-1,1)
currentRegMotorValues = []
currentRegCurve = []
fd_current = []


for i in range(1,9):
    filtered = fd[fd[f'I{i}']> fd[f'I{i}'].min()]
    filtered = filtered[filtered[f'rpm{i}']> 0]
    X = filtered[[f'rpm{i}']]
    Y = filtered[[f'I{i}']]
    features = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X)
    currentRegression = LinearRegression(fit_intercept=False, positive=True)
    currentRegression.fit(features, Y)
    current_range_poly = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm_range)
    reg_curve = currentRegression.predict(current_range_poly)
    currentRegMotorValues.append(reg_curve)
    fd_current.append(filtered)
    currentRegCurve.append(currentRegression)

In [ ]:
# correct values for current by using regression model

# save value of flight data DataFrame
fd_corrected = fd.copy(deep = True)
fd_save = fd.copy(deep = True)

# get indices where I3 is minimum
I3Indices = fd['I3']==fd[f'I3'].min()
rpm3Data = fd['rpm3'][I3Indices].values.reshape(-1,1)

# trasnform polynomial features
rpm3Features = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm3Data)

# perform regression
rpm3RegressionResult = currentRegCurve[2].predict(rpm3Features).reshape(-1)

# apply correction to current signal
fd_corrected.loc[I3Indices,'I3'] = rpm3RegressionResult

In [ ]:
figCurrent, axCurrent = plt.subplots(1,2, figsize=(10,5))
figCurrent.suptitle("Correction of current signal", fontsize=14)

axCurrent[0].plot(fd['t'], fd['I3'], color=upCol, label='current3', zorder=1)
axCurrent[0].scatter(fd['t'][I3Indices], fd['I3'][I3Indices], color='r', s=1, label='current3')

axCurrent[1].plot(fd_corrected['t'], fd_corrected['I3'], color=upCol, label='current3 (corrected)', zorder=1)
axCurrent[1].scatter(fd_corrected['t'][I3Indices], fd_corrected['I3'][I3Indices], color='r', s=1, label='current3')

for i in range(2):
    axCurrent[i].set_xlabel('Time [s]')
    axCurrent[i].set_ylabel('Current [A]')
    axCurrent[i].set_ylim(0,20)
    axCurrent[i].grid()
    axCurrent[i].legend()


In [ ]:
# Apply current correction
fd = fd_corrected

In [ ]:
# Reset current correction
#fd = fd_save

In [ ]:
for i in range(1,9):
    fd[f'pMo{i}'] = fd[f'U{i}'] * fd[f'I{i}']

# delta RPM for each individual arm
fd[f'dRpmArm1'] = fd['rpm1'] - fd['rpm6']
fd[f'dRpmArm2'] = fd['rpm2'] - fd['rpm5']
fd[f'dRpmArm3'] = fd['rpm3'] - fd['rpm8']
fd[f'dRpmArm4'] = fd['rpm4'] - fd['rpm7']

# add power for each individual arm
fd[f'pArm1'] = fd['pMo1'] + fd['pMo6']
fd[f'pArm2'] = fd['pMo2'] + fd['pMo5']
fd[f'pArm3'] = fd['pMo3'] + fd['pMo8']
fd[f'pArm4'] = fd['pMo4'] + fd['pMo7']

# calculate total vehicle power
fd['pVehicle'] = fd['pArm1'] + fd['pArm2'] + fd['pArm3'] + fd['pArm4']

### Outlier removal

In [ ]:
%%capture

deltaList = []
boxLabels = []
for d in np.unique(delta0):
    deltaList.append(np.intersect1d(np.where(delta0== d), hover))
    boxLabels.append(f'{d}')

boxArm1 = [power_pair['25'].values[delta] for delta in deltaList]
boxArm2 = [power_pair['16'].values[delta] for delta in deltaList]
boxArm3 = [power_pair['38'].values[delta] for delta in deltaList]
boxArm4 = [power_pair['47'].values[delta] for delta in deltaList]
boxTotal = [boxArm1[i] + boxArm2[i] + boxArm3[i] + boxArm4[i] for i in range(len(deltaList))]

# get indices of all outliers

boxArms = [boxArm1, boxArm2, boxArm3, boxArm4]
powerPairs = [power_pair['25'].values, power_pair['16'].values, power_pair['38'].values, power_pair['47'].values]
outliers = np.empty(0, int)

for i in range(4):  # iterate over all 4 arms
    
    box = plt.boxplot(boxArms[i])

    for j in range (len(box['fliers'])):
        fliers = box['fliers'][j].get_ydata()
        indices = np.where(np.isin(powerPairs[i], fliers) == True)
        outliers = np.append(outliers, indices)

outliers = np.unique(outliers)

# use this hover_new variable instead of hover to remove outliers (does not seem to have much impact)
hover_new = np.setdiff1d(hover, outliers)

## Start of analysis

In [ ]:
# plot histogram of vnorm with corresponding limit

fig0, ax0 = plt.subplots(1, 2, figsize=(10,5))
fig0.suptitle('Velocity profile', fontsize=14)

# vnorm in percentage of v_i
v_norm_percentage = (v_norm/v_induced)*100  

# plot moving average of vertical velocity w and highlight the hovering sequence
ax0[0].plot(time, moving_average(w), label='w full flight')
ax0[0].plot(time[hover], moving_average(w[hover]), label='w hover sequence')
#ax0[0].plot(time[hover_new], moving_average(w[hover_new]), label='w hover outlier')

# plot v_norm and hightlight hover sequence
ax0[1].plot(time, v_norm_percentage, label='v_norm')
ax0[1].plot(time[hover], v_norm_percentage[hover], label='v_norm hover sequence')

# add red boundary lines to highlight separation between take-off/landing and hovering
ax0[1].plot([hover_start, hover_start],[0, 3], 'r--')
ax0[1].plot([max(time)-hover_end, max(time)-hover_end],[0, 3], 'r--')

# add grid, legend, labels, etc.
ax0[0].grid()
ax0[0].legend()
ax0[0].set_title('Vertical velocity')
ax0[0].set_xlim(min(time), max(time))

ax0[1].set_title('v norm')
ax0[1].set_xlabel('time [s]')
ax0[1].set_ylabel('v_norm [% of $v_i$]')
ax0[1].grid()
ax0[1].legend()
ax0[1].set_xlim(min(time),max(time))


In [ ]:
# Plot control commands

fig1, ax1 = plt.subplots(2, 2, figsize=(10,8))
fig1.suptitle('Control commands', fontsize=14)

ax1[0][0].plot(time,control[0], label='roll cmd')
ax1[0][1].plot(time,control[1], label='pitch cmd')
ax1[1][0].plot(time,control[2], label='yaw cmd')
ax1[1][1].plot(time,control[3], label='throttle cmd')

for i in range(2):
    for j in range(2):
        ax1[i][j].set_xlabel('Time [s]')
        ax1[i][j].set_ylim(-1,1)
        ax1[i][j].grid()
        ax1[i][j].legend()

ax1[0][0].set_ylabel(f'Roll cmd')
ax1[0][1].set_ylabel(f'Pitch cmd')
ax1[1][0].set_ylabel(f'Yaw cmd')
ax1[1][1].set_ylabel(f'Throttle')

ax1[0][0].set_title(f'Roll cmd')
ax1[0][1].set_title(f'Pitch cmd')
ax1[1][0].set_title(f'Yaw cmd')
ax1[1][1].set_title(f'Throttle')

fig1.tight_layout()

In [ ]:
# Plot motor cmds

fig2, ax2 = plt.subplots(2, 2, figsize=(10,8))
fig2.suptitle('Motor commands from B_plus matrix', fontsize=14)

# Rotor pairs

ax2[0][0].plot(time,delta_cmd[1,:], color=upCol, label='rotor2 (upper)')
ax2[0][0].plot(time,delta_cmd[4,:], color=lowCol, label='rotor5 (lower)')

ax2[0][1].plot(time,delta_cmd[0,:], color=upCol, label='rotor1 (upper)')
ax2[0][1].plot(time,delta_cmd[5,:], color=lowCol, label='rotor6 (lower)')


ax2[1][0].plot(time,delta_cmd[2,:], color=upCol, label='rotor3 (upper)')
ax2[1][0].plot(time,delta_cmd[7,:], color=lowCol, label='rotor8 (lower)')

ax2[1][1].plot(time,delta_cmd[3,:], color=upCol, label='rotor4 (upper)')
ax2[1][1].plot(time,delta_cmd[6,:], color=lowCol, label='rotor7 (lower)')


for i in range(2):
    for j in range(2):
        ax2[i][j].set_xlabel('Time [s]')
        ax2[i][j].set_ylabel('$\delta$')
        ax2[i][j].set_ylim(-3,3)
        ax2[i][j].grid()
        ax2[i][j].legend()

ax2[0][0].set_title(f'Motor command 2-5')
ax2[0][1].set_title(f'Motor command 1-6')
ax2[1][0].set_title(f'Motor command 3-8')
ax2[1][1].set_title(f'Motor command 4-7')
fig2.tight_layout()

In [ ]:
# Plot RPM over time

fig3, ax3 = plt.subplots(2, 2, figsize=(10,8))
fig3.suptitle('RPM motors', fontsize=14)

# Rotor pairs

ax3[0][0].plot(time,omega[1], color=upCol, label='rotor2 (upper)')
ax3[0][0].plot(time,omega[4], color=lowCol,label='rotor5 (lower)')

ax3[0][1].plot(time,omega[0], color=upCol, label='rotor1 (upper)')
ax3[0][1].plot(time,omega[5], color=lowCol, label='rotor6 (lower)')


ax3[1][0].plot(time,omega[2], color=upCol, label='rotor3 (upper)')
ax3[1][0].plot(time,omega[7], color=lowCol, label='rotor8 (lower)')

ax3[1][1].plot(time,omega[3], color=upCol, label='rotor4 (upper)')
ax3[1][1].plot(time,omega[6], color=lowCol, label='rotor7 (lower)')


for i in range(2):
    for j in range(2):
        ax3[i][j].set_xlabel('Time [s]')
        ax3[i][j].set_ylabel('RPM')
        ax3[i][j].grid()
        ax3[i][j].legend()

ax3[0][0].set_title(f'RPM Motor pair 2-5')
ax3[0][1].set_title(f'RPM Motor pair 1-6')
ax3[1][0].set_title(f'RPM Motor pair 3-8')
ax3[1][1].set_title(f'RPM Motor pair 4-7')

fig3.tight_layout()

In [ ]:
# delta RPM

fig4, ax4 = plt.subplots(2, 2, figsize=(10,8))
fig4.suptitle('Delta RPM', fontsize=14)

# Rotor pairs

ax4[0][0].plot(time,omega[1] - omega[4], label='$\Delta$-RPM pair 2-5')
ax4[0][1].plot(time,omega[0] - omega[5], label='$\Delta$-RPM pair 1-6')
ax4[1][0].plot(time,omega[2] - omega[7], label='$\Delta$-RPM pair 3-8')
ax4[1][1].plot(time,omega[3] - omega[6], label='$\Delta$-RPM pair 4-7')

for i in range(2):
    for j in range(2):
        ax4[i][j].set_xlabel('Time [s]')
        ax4[i][j].set_ylabel('RPM')
        ax4[i][j].grid()
        ax4[i][j].legend()

ax4[0][0].set_title(f'RPM Motor pair 2-5')
ax4[0][1].set_title(f'RPM Motor pair 1-6')
ax4[1][0].set_title(f'RPM Motor pair 3-8')
ax4[1][1].set_title(f'RPM Motor pair 4-7')

fig4.tight_layout()

In [ ]:
# Plot current over time

fig5, ax5 = plt.subplots(2, 2, figsize=(10,8))
fig5.suptitle('Motor current', fontsize=14)
# Rotor pairs

ax5[0][0].plot(fd['t'],fd['I2'], color=upCol, label='rotor2 (upper)')
ax5[0][0].plot(fd['t'],fd['I5'], color=lowCol, label='rotor5 (lower)')

ax5[0][1].plot(fd['t'],fd['I1'], color=upCol, label='rotor1 (upper)')
ax5[0][1].plot(fd['t'],fd['I6'], color=lowCol, label='rotor6 (lower)')


ax5[1][0].plot(fd['t'],fd['I3'], color=upCol, label='rotor3 (upper)', zorder=1)
ax5[1][0].plot(fd['t'],fd['I8'], color=lowCol, label='rotor8 (lower)')

ax5[1][1].plot(fd['t'],fd['I4'], color=upCol, label='rotor4 (upper)')
ax5[1][1].plot(fd['t'],fd['I7'], color=lowCol, label='rotor7 (lower)')


# plot "problem" points for motor 3



for i in range(2):
    for j in range(2):
        ax5[i][j].set_xlabel('Time [s]')
        ax5[i][j].set_ylabel('Current [A]')
        ax5[i][j].set_ylim(0,20)
        ax5[i][j].grid()
        ax5[i][j].legend()

ax5[0][0].set_title(f'Current Motor pair 2-5')
ax5[0][1].set_title(f'Current Motor pair 1-6')
ax5[1][0].set_title(f'Current Motor pair 3-8')
ax5[1][1].set_title(f'Current Motor pair 4-7')

#ax5[1][0].scatter(fd['t'][fd['I3'] == fd['I3'].min()],fd['I3'][fd['I3'] == fd['I3'].min()],
#                  color='k', s=15, marker='o', label='faulty current')

fig5.tight_layout()

In [ ]:
# voltage over time

fig6, ax6 = plt.subplots(2, 2, figsize=(10,8))
fig6.suptitle('Motor voltage', fontsize=14)
# Rotor pairs

ax6[0][0].plot(fd['t'], fd['U2'], color=upCol, label='rotor2 (upper)')
ax6[0][0].plot(fd['t'], fd['U5'], color=lowCol, label='rotor5 (lower)')

ax6[0][1].plot(fd['t'], fd['U1'], color=upCol, label='rotor1 (upper)')
ax6[0][1].plot(fd['t'], fd['U6'], color=lowCol, label='rotor6 (lower)')


ax6[1][0].plot(fd['t'], fd['U3'], color=upCol, label='rotor3 (upper)')
ax6[1][0].plot(fd['t'], fd['U8'], color=lowCol, label='rotor8 (lower)')

ax6[1][1].plot(fd['t'], fd['U4'], color=upCol, label='rotor4 (upper)')
ax6[1][1].plot(fd['t'], fd['U7'], color=lowCol, label='rotor7 (lower)')


for i in range(2):
    for j in range(2):
        ax6[i][j].set_xlabel('Time [s]')
        ax6[i][j].set_ylabel('Voltage [V]')
        ax6[i][j].set_ylim(27, 33.5)
        ax6[i][j].grid()
        ax6[i][j].legend()

ax6[0][0].set_title(f'Voltage Motor pair 2-5')
ax6[0][1].set_title(f'Voltage Motor pair 1-6')
ax6[1][0].set_title(f'Voltage Motor pair 3-8')
ax6[1][1].set_title(f'Voltage Motor pair 4-7')

fig6.tight_layout()

In [ ]:
# Power over time for each motor

fig7, ax7 = plt.subplots(2, 2, figsize=(10,8))
fig7.suptitle('Motor power', fontsize=14)

# Rotor pairs

ax7[0][0].plot(fd['t'], fd['pMo2'], color=upCol, label='rotor2 (upper)')
ax7[0][0].plot(fd['t'], fd['pMo5'], color=lowCol, label='rotor5 (lower)')

ax7[0][1].plot(fd['t'], fd['pMo1'], color=upCol, label='rotor1 (upper)')
ax7[0][1].plot(fd['t'], fd['pMo6'], color=lowCol, label='rotor6 (lower)')


ax7[1][0].plot(fd['t'], fd['pMo3'], color=upCol, label='rotor3 (upper)')
ax7[1][0].plot(fd['t'], fd['pMo8'], color=lowCol, label='rotor8 (lower)')

ax7[1][1].plot(fd['t'], fd['pMo4'], color=upCol, label='rotor4 (upper)')
ax7[1][1].plot(fd['t'], fd['pMo7'], color=lowCol, label='rotor7 (lower)')


for i in range(2):
    for j in range(2):
        ax7[i][j].set_xlabel('Time [s]')
        ax7[i][j].set_ylabel('Power [W]')
        ax7[i][j].set_ylim(0,700)
        ax7[i][j].grid()
        ax7[i][j].legend()

ax7[0][0].set_title(f'Power Motor pair 2-5')
ax7[0][1].set_title(f'Power Motor pair 1-6')
ax7[1][0].set_title(f'Power Motor pair 3-8')
ax7[1][1].set_title(f'Power Motor pair 4-7')

fig7.tight_layout()

In [ ]:
# Perform regression Power vs. RPM

rpm_range = np.linspace(0, 3000, 100).reshape(-1,1)
rpmRegMotorFiltered = []
fd_filtered = []


for i in range(1,9):
    filtered = fd[fd[f'I{i}']> fd[f'I{i}'].min()]
    filtered = filtered[filtered[f'rpm{i}']> 0]
    X = filtered[[f'rpm{i}']]
    Y = filtered[[f'pMo{i}']]
    features = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X)
    rpmRegression = LinearRegression(fit_intercept=False, positive=True)
    rpmRegression.fit(features, Y)
    rpm_range_poly = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm_range)
    reg_curve = rpmRegression.predict(rpm_range_poly)
    rpmRegMotorFiltered.append(reg_curve)
    fd_filtered.append(filtered)

In [ ]:
# correct values for current by using regression model

# save value of flight data DataFrame
#fd_corrected = fd.copy(deep = True)
#fd_save = fd.copy(deep = True)

# get rpm values where I3 is minimum
#rpm3Data = fd['rpm3'][fd[f'I3']==fd[f'I3'].min()].values.reshape(-1,1)

# trasnform polynomial features
#rpm3Features = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm3Data)

# perform regression
#rpm3RegressionResult = currentRegCurve[2].predict(rpm3Features).reshape(-1)

# apply correction to current signal
#I3Indices = fd['I3']==fd[f'I3'].min()
#fd_corrected.loc[I3Indices,'I3'] = rpm3RegressionResult

In [ ]:
# Power over effective RPM

fig8, ax8 = plt.subplots(2, 2, figsize=(10,8))
fig8.suptitle('Power vs. RPM', fontsize=14)

# Rotor pairs

ax8[0][0].scatter(fd['rpm2'], fd['pMo2'], s=markersize, color=upCol, label='rotor2 (upper)')
ax8[0][0].scatter(fd['rpm5'], fd['pMo5'], s=markersize, color=lowCol, label='rotor5 (lower)')

ax8[0][1].scatter(fd['rpm1'], fd['pMo1'], s=markersize, color=upCol, label='rotor1 (upper)')
ax8[0][1].scatter(fd['rpm6'], fd['pMo6'], s=markersize, color=lowCol, label='rotor6 (lower)')

ax8[1][0].scatter(fd['rpm3'], fd['pMo3'], s=markersize, color=upCol, label='rotor3 (upper)')
ax8[1][0].scatter(fd['rpm8'], fd['pMo8'], s=markersize, color=lowCol, label='rotor8 (lower)')

ax8[1][1].scatter(fd['rpm4'], fd['pMo4'], s=markersize, color=upCol, label='rotor4 (upper)')
ax8[1][1].scatter(fd['rpm7'], fd['pMo7'], s=markersize, color=lowCol, label='rotor7 (lower)')

ax8[0][0].plot(rpm_range, rpmRegMotorFiltered[1], linestyle='dashed', color=upMeanCol, label='rotor5 (lower)')
ax8[0][0].plot(rpm_range, rpmRegMotorFiltered[4], linestyle='dashed', color=lowMeanCol, label='rotor5 (lower)')

ax8[0][1].plot(rpm_range, rpmRegMotorFiltered[0], linestyle='dashed', color=upMeanCol, label='rotor1 (lower)')
ax8[0][1].plot(rpm_range, rpmRegMotorFiltered[5], linestyle='dashed', color=lowMeanCol, label='rotor6 (lower)')

ax8[1][0].plot(rpm_range, rpmRegMotorFiltered[2], linestyle='dashed', color=upMeanCol, label='rotor3 (lower)')
ax8[1][0].plot(rpm_range, rpmRegMotorFiltered[7], linestyle='dashed', color=lowMeanCol, label='rotor8 (lower)')

ax8[1][1].plot(rpm_range, rpmRegMotorFiltered[3], linestyle='dashed', color=upMeanCol, label='rotor4 (lower)')
ax8[1][1].plot(rpm_range, rpmRegMotorFiltered[6], linestyle='dashed', color=lowMeanCol, label='rotor7 (lower)')

for i in range(2):
    for j in range(2):
        ax8[i][j].set_xlabel('RPM')
        ax8[i][j].set_ylabel('Power [W]')
        ax8[i][j].set_xlim(0,4000)
        ax8[i][j].set_ylim(0,600)
        ax8[i][j].grid()
        ax8[i][j].legend()

ax8[0][0].set_title(f'Power vs. RPM motor-pair 2-5')
ax8[0][1].set_title(f'Power vs. RPM motor-pair 1-6')
ax8[1][0].set_title(f'Power vs. RPM motor-pair 3-8')
ax8[1][1].set_title(f'Power vs. RPM motor-pair 4-7')

fig8.tight_layout()

In [ ]:
# Power per motor over delta0

fig9, ax9 = plt.subplots(2, 2, figsize=(10,8))
fig9.suptitle('Power vs. $\Delta_{0}$ per motor', fontsize=14)

# Rotor pairs
ax9[0][0].scatter(delta0[hover], power[1].values[hover], s=markersize, color= upCol, label='rotor2 (upper)')
ax9[0][0].scatter(delta0_values, meanPowerDelta0[1], marker='x', color=upMeanCol, s=50, label='rotor2 mean')

ax9[0][0].scatter(delta0[hover], power[4].values[hover], s=markersize, color= lowCol, label='rotor5 (lower)')
ax9[0][0].scatter(delta0_values, meanPowerDelta0[4], marker='x', color=lowMeanCol, s=50, label='rotor5 mean')

ax9[0][1].scatter(delta0[hover], power[0].values[hover], s=markersize, color= upCol, label='rotor1 (upper)')
ax9[0][1].scatter(delta0_values, meanPowerDelta0[0], marker='x', color=upMeanCol, s=50, label='rotor1 mean')

ax9[0][1].scatter(delta0[hover], power[5].values[hover], s=markersize, color= lowCol, label='rotor6 (lower)')
ax9[0][1].scatter(delta0_values, meanPowerDelta0[5], marker='x', color=lowMeanCol, s=50, label='rotor6 mean')

ax9[1][0].scatter(delta0[hover], power[2].values[hover], s=markersize, color= upCol, label='rotor3 (upper)')
ax9[1][0].scatter(delta0_values, meanPowerDelta0[2], marker='x', color=upMeanCol, s=50, label='rotor3 mean')

ax9[1][0].scatter(delta0[hover], power[7].values[hover], s=markersize, color= lowCol, label='rotor8 (lower)')
ax9[1][0].scatter(delta0_values, meanPowerDelta0[7], marker='x', color=lowMeanCol, s=50, label='rotor8 mean')

ax9[1][1].scatter(delta0[hover], power[3].values[hover], s=markersize, color= upCol, label='rotor4 (upper)')
ax9[1][1].scatter(delta0_values, meanPowerDelta0[3], marker='x', color=upMeanCol, s=50, label='rotor4 mean')

ax9[1][1].scatter(delta0[hover], power[6].values[hover], s=markersize, color= lowCol, label='rotor7 (lower)')
ax9[1][1].scatter(delta0_values, meanPowerDelta0[6], marker='x', color=lowMeanCol, s=50, label='rotor7 mean')

#
for i in range(2):
    for j in range(2):
        ax9[i][j].set_xlabel('$\Delta_{0}$')
        ax9[i][j].set_ylabel('Power [W]')
        ax9[i][j].set_ylim(0,600)
        ax9[i][j].grid()
        ax9[i][j].legend()

ax9[0][0].set_title(f'Motor pair 2-5: Power vs. $\Delta_0$')
ax9[0][1].set_title(f'Motor pair 1-6: Power vs. $\Delta_0$')
ax9[1][0].set_title(f'Motor pair 3-8: Power vs. $\Delta_0$')
ax9[1][1].set_title(f'Motor pair 4-7: Power vs. $\Delta_0$')
fig9.tight_layout()

In [ ]:
# Power per arm over delta0

fig10, ax10 = plt.subplots(2, 2, figsize=(10,8))
fig10.suptitle('Power vs. $\Delta_{0}$ per arm', fontsize=14)

# Arms
ax10[0][0].scatter(delta0[hover], power_pair['25'].values[hover], color=armCol, s=markersize, label='arm1')
ax10[0][1].scatter(delta0[hover], power_pair['16'].values[hover], color=armCol, s=markersize, label='arm2')
ax10[1][0].scatter(delta0[hover], power_pair['38'].values[hover], color=armCol, s=markersize, label='arm3')
ax10[1][1].scatter(delta0[hover], power_pair['47'].values[hover], color=armCol, s=markersize, label='arm4')

# Rotor pairs
markersize=50
ax10[0][0].scatter(delta0_values, meanPowerArm[0], marker='x', color=armMeanCol, s=markersize, label='arm1 mean')
ax10[0][1].scatter(delta0_values, meanPowerArm[1], marker='x', color=armMeanCol, s=markersize, label='arm2 mean')
ax10[1][0].scatter(delta0_values, meanPowerArm[2], marker='x', color=armMeanCol, s=markersize, label='arm3 mean')
ax10[1][1].scatter(delta0_values, meanPowerArm[3], marker='x', color=armMeanCol, s=markersize, label='arm4 mean')

markersize=15

for i in range(2):
    for j in range(2):
        ax10[i][j].set_xlabel('$\Delta_{0}$-cmd')
        ax10[i][j].set_ylabel('Power [W]')
        ax10[i][j].set_ylim(0, 900)
        ax10[i][j].grid()
        ax10[i][j].legend()

ax10[0][0].set_title(f'Arm 1 (motors 2-5): Power vs. $\Delta_0$')
ax10[0][1].set_title(f'Arm 2 (motors 1-6): Power vs. $\Delta_0$')
ax10[1][0].set_title(f'Arm 3 (motors 3-8): Power vs. $\Delta_0$')
ax10[1][1].set_title(f'Arm 3 (motors 4-7): Power vs. $\Delta_0$')
fig10.tight_layout()

In [ ]:
# Power per arm over delta0

fig11, ax11 = plt.subplots(2, 2, figsize=(10,8))
fig11.suptitle('Boxplot Power vs. $\Delta_{0}$ per arm', fontsize=14)

# Arms
ax11[0][0].boxplot(boxArm1)
ax11[0][1].boxplot(boxArm2)
ax11[1][0].boxplot(boxArm3)
ax11[1][1].boxplot(boxArm4)

for i in range(2):
    for j in range(2):
        ax11[i][j].set_xlabel('$\Delta_{0}$-cmd')
        ax11[i][j].set_xticklabels(boxLabels)
        ax11[i][j].grid()

fig11.tight_layout()

In [ ]:
# Power per arm over deltaRPM

fig12, ax12 = plt.subplots(2, 2, figsize=(10,8))
fig12.suptitle('Delta-RPM vs. $\Delta_{0}$ per arm', fontsize=14)

# Arms
ax12[0][0].scatter(delta0[hover], deltaRPM['25'].values[hover], color=armCol, s=markersize, label='arm1')
ax12[0][1].scatter(delta0[hover], deltaRPM['16'].values[hover], color=armCol, s=markersize, label='arm2')
ax12[1][0].scatter(delta0[hover], deltaRPM['38'].values[hover], color=armCol, s=markersize, label='arm3')
ax12[1][1].scatter(delta0[hover], deltaRPM['47'].values[hover], color=armCol, s=markersize, label='arm4')

# Rotor pairs
markersize=50
ax12[0][0].scatter(delta0_values, deltaRpmArm[0], marker='x', color=armMeanCol, s=markersize, label='mean ')
ax12[0][1].scatter(delta0_values, deltaRpmArm[1], marker='x', color=armMeanCol, s=markersize, label='mean Power')
ax12[1][0].scatter(delta0_values, deltaRpmArm[2], marker='x', color=armMeanCol, s=markersize, label='mean Power')
ax12[1][1].scatter(delta0_values, deltaRpmArm[3], marker='x', color=armMeanCol, s=markersize, label='mean Power')

markersize=15

for i in range(2):
    for j in range(2):
        ax12[i][j].set_xlabel('$\Delta_{0}$-cmd')
        ax12[i][j].set_ylabel('Delta-RPM [-]')
        ax12[i][j].set_ylim(-2000, 500)
        ax12[i][j].grid()
        ax12[i][j].legend(loc='upper left')

ax12[0][0].set_title(f'Arm 1 (motors 2-5): Delta-RPM vs. $\Delta_0$')
ax12[0][1].set_title(f'Arm 2 (motors 1-6): Delta-RPM vs. $\Delta_0$')
ax12[1][0].set_title(f'Arm 3 (motors 3-8): Delta-RPM vs. $\Delta_0$')
ax12[1][1].set_title(f'Arm 3 (motors 4-7): Delta-RPM vs. $\Delta_0$')
fig12.tight_layout()

In [ ]:
# Power per arm over deltaRPM

fig13, ax13 = plt.subplots(2, 2, figsize=(10,8))
fig13.suptitle('Power vs. actual delta RPM', fontsize=14)

# Arms
ax13[0][0].scatter(fd['dRpmArm1'].values[hover], fd['pArm1'].values[hover], color=armCol, s=markersize, label='arm1')
ax13[0][1].scatter(fd['dRpmArm2'].values[hover], fd['pArm2'].values[hover], color=armCol, s=markersize, label='arm2')
ax13[1][0].scatter(fd['dRpmArm3'].values[hover], fd['pArm3'].values[hover], color=armCol, s=markersize, label='arm3')
ax13[1][1].scatter(fd['dRpmArm4'].values[hover], fd['pArm4'].values[hover], color=armCol, s=markersize, label='arm4')

for i in range(2):
    for j in range(2):
        ax13[i][j].set_xlabel('Delta-RPM [-]')
        ax13[i][j].set_ylabel('Power [W]')
        ax13[i][j].set_ylim(0, 600)
        ax13[i][j].grid()
        ax13[i][j].legend(loc='upper left')

ax13[0][0].set_title(f'Arm 1 (motors 2-5): Power vs. Delta-RPM')
ax13[0][1].set_title(f'Arm 2 (motors 1-6): Power vs. Delta-RPM')
ax13[1][0].set_title(f'Arm 3 (motors 3-8): Power vs. Delta-RPM')
ax13[1][1].set_title(f'Arm 3 (motors 4-7): Power vs. Delta-RPM')
fig13.tight_layout()

In [ ]:
fig14, ax14 = plt.subplots(1,2,figsize=(10,4))
fig14.suptitle('Total vehicle power over $\Delta_0$ and Delta-RPM', fontsize=14)

ax14[0].scatter(delta0[hover], total_power.values[hover], color=armCol, s=markersize, label='total vehicle power')
ax14[0].scatter(delta0_values, vehiclePower, marker='x', s=50, color=armMeanCol, label='mean vehicle power')
ax14[0].scatter(delta0_values, medianVehiclePower, marker='x', s=50, color='b', label='median vehicle power')

ax14[1].scatter(fd['dRpmArm1'].values[hover], fd['pVehicle'].values[hover], color=armCol, s=markersize, label='total vehicle power')

ax14[0].set_xlabel('$\Delta_{0}$-cmd')
ax14[0].set_ylabel('Power [W]')
ax14[0].set_ylim(1000, 2100)
ax14[0].grid()
ax14[0].legend(loc='lower left')
ax14[0].set_title('Total vehicle power vs. $\Delta_0$')    

ax14[1].set_xlabel('Delta-RPM')
ax14[1].set_ylabel('Power [W]')
ax14[1].set_ylim(1000, 2000)
ax14[1].grid()
ax14[1].set_title('Total vehicle power vs. Delta-RPM') 

fig14.tight_layout()

## Model bulding (will be moved to separate script)

### Analyse correlation between power and velocity signal

In [ ]:
# Total power over time

fig15, ax15 = plt.subplots(2,2, figsize=(10,8))
fig15.suptitle('Comparison of power and velocity signal', fontsize=14)
# Rotor pairs

# vetical velocity plot (raw and smoothed with moving average filter)
ax15[0][0].set_title(f'Vertical velocity w during hover')
ax15[0][0].plot(time[hover], w[hover], label='w')
ax15[0][0].plot(time[hover], moving_average(w,300)[hover], label='w (smoothed)')

# total power (sum of all motors), raw and smoothed
ax15[0][1].set_title(f'Total power during hover')
ax15[0][1].plot(time[hover], total_power.values[hover], label='Total power')
ax15[0][1].plot(time[hover], moving_average(total_power.values,300)[hover], label='Total power (smoothed)')

# z-normalized signal of power and velocity
normalized_power = normalization(moving_average(total_power.values,20)[hover])
normalized_velocity = normalization(moving_average(z,20)[hover])

# plot normalized signals
ax15[1][0].set_title(f'z-normalized signals')
ax15[1][0].plot(time[hover], normalized_power, label='normalized power')
ax15[1][0].plot(time[hover], normalized_velocity, label='normalized w')

# Corrected signal
ax15[1][1].set_title(f'Correction of power signal')
ax15[1][1].plot(time[hover], normalized_power, label='normalized power')
ax15[1][1].plot(time[hover], normalized_power + normalized_velocity/2, label='corrected power')
ax15[1][1].plot(time[hover], delta0[hover], label='$\Delta_0$')


for i in range(2):
        ax15[0][i].set_xlabel('Time [s]')
        ax15[0][i].grid()
        ax15[0][i].legend()

for i in range(2):
    ax15[1][i].grid()
    ax15[1][i].set_xlabel('Time [s]')
    ax15[1][i].legend()

ax15[0][0].set_ylabel('Velocity [m/s]')
ax15[0][1].set_ylabel('Power [W]')
ax15[1][0].set_ylabel('[-]')
ax15[1][1].set_ylabel('[-]')

fig15.tight_layout()

### Distribution for $\Delta_0$-values

In [ ]:
# analyse distribution for each delta0

fig16, ax16 = plt.subplots(1,1,figsize=(5,4))

value = total_power[delta0==-0.4].values

n, bins, _ = ax16.hist(value, bins=100)

ax16.plot([np.mean(value), np.mean(value)],[0, max(n)], 'r--', label='mean')
ax16.plot([np.median(value), np.median(value)],[0, max(n)], 'k--', label='median')
ax16.grid()
ax16.legend()
fig16.tight_layout()

### Correlation analysis

In [ ]:
df_corr = fd.drop(columns={'t'})
df_corr = df_corr.rolling(30).mean()
df_corr = df_corr.dropna()

for i in range(1,9):
    df_corr = df_corr.drop(columns={f'U{i}'})
    df_corr = df_corr.drop(columns={f'I{i}'})
    df_corr = df_corr.drop(columns={f'rpm{i}'})
    df_corr = df_corr.drop(columns={f'uOut{i}'})
    
    if i < 5:
        df_corr = df_corr.drop(columns={f'uIn{i}'})


df_corr = df_corr.corr()


colormap = plt.cm.RdBu

fig17, ax17 = plt.subplots(figsize=(10,5))

fig17.suptitle(u'Crosscorrelation', y=1.05, size=16)

mask = np.zeros_like(df_corr.iloc[0:30,0:30])
mask[np.triu_indices_from(mask)] = True

svm = sns.heatmap(df_corr.iloc[0:30,0:30], mask=mask, linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=False)

In [ ]:
# Perform regression RPM vs. current

#rpm_range = np.linspace(0, 3000, 100).reshape(-1,1)
#currentRegMotorValues = []
#currentRegCurve = []
#fd_current = []


#for i in range(1,9):
    #filtered = fd[fd[f'I{i}']> fd[f'I{i}'].min()]
    #filtered = filtered[filtered[f'rpm{i}']> 0]
    #X = filtered[[f'rpm{i}']]
    #Y = filtered[[f'I{i}']]
    #features = PolynomialFeatures(degree=3, include_bias=False).fit_transform(X)
    #currentRegression = LinearRegression(fit_intercept=False, positive=True)
    #currentRegression.fit(features, Y)
    #current_range_poly = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm_range)
    #reg_curve = currentRegression.predict(current_range_poly)
    #currentRegMotorValues.append(reg_curve)
    #fd_current.append(filtered)
    #currentRegCurve.append(currentRegression)

In [ ]:
# correct values for current by using regression model

# save value of flight data DataFrame
#fd_corrected = fd.copy(deep = True)
#fd_save = fd.copy(deep = True)

# get rpm values where I3 is minimum
#rpm3Data = fd['rpm3'][fd[f'I3']==fd[f'I3'].min()].values.reshape(-1,1)

# trasnform polynomial features
#rpm3Features = PolynomialFeatures(degree=3, include_bias=False).fit_transform(rpm3Data)

# perform regression
#rpm3RegressionResult = currentRegCurve[2].predict(features).reshape(-1)

# apply correction to current signal
#I3Indices = fd['I3']==fd[f'I3'].min()
#fd_corrected.loc[I3Indices,'I3'] = rpm3RegressionResult

In [ ]:
## Current over RPM
#fd = fd_corrected
#fig8, ax8 = plt.subplots(2, 2, figsize=(10,8))
#fig8.suptitle('Power vs. RPM', fontsize=14)
#
## Rotor pairs
#
#ax8[0][0].scatter(fd['rpm2'], fd['I2'], s=markersize, color=upCol, label='rotor2 (upper)')
#ax8[0][0].scatter(fd['rpm5'], fd['I5'], s=markersize, color=lowCol, label='rotor5 (lower)')
#
#ax8[0][1].scatter(fd['rpm1'], fd['I1'], s=markersize, color=upCol, label='rotor1 (upper)')
#ax8[0][1].scatter(fd['rpm6'], fd['I6'], s=markersize, color=lowCol, label='rotor6 (lower)')
#
#ax8[1][0].scatter(fd['rpm3'], fd['I3'], s=markersize, color=upCol, label='rotor3 (upper)')
##ax8[1][0].scatter(fd['rpm8'], fd['I8'], s=markersize, color=lowCol, label='rotor8 (lower)')
#
#ax8[1][1].scatter(fd['rpm4'], fd['I4'], s=markersize, color=upCol, label='rotor4 (upper)')
#ax8[1][1].scatter(fd['rpm7'], fd['I7'], s=markersize, color=lowCol, label='rotor7 (lower)')
#
#ax8[0][0].plot(rpm_range, currentRegMotorValues[1], linestyle='dashed', color=upMeanCol, label='rotor5 (lower)')
#ax8[0][0].plot(rpm_range, currentRegMotorValues[4], linestyle='dashed', color=lowMeanCol, label='rotor5 (lower)')
#
#ax8[0][1].plot(rpm_range, currentRegMotorValues[0], linestyle='dashed', color=upMeanCol, label='rotor1 (lower)')
#ax8[0][1].plot(rpm_range, currentRegMotorValues[5], linestyle='dashed', color=lowMeanCol, label='rotor6 (lower)')
#
#ax8[1][0].plot(rpm_range, currentRegMotorValues[2], linestyle='dashed', color=upMeanCol, label='rotor3 (lower)')
#ax8[1][0].plot(rpm_range, currentRegMotorValues[7], linestyle='dashed', color=lowMeanCol, label='rotor8 (lower)')
#
#ax8[1][1].plot(rpm_range, currentRegMotorValues[3], linestyle='dashed', color=upMeanCol, label='rotor4 (lower)')
#ax8[1][1].plot(rpm_range, currentRegMotorValues[6], linestyle='dashed', color=lowMeanCol, label='rotor7 (lower)')
#
#for i in range(2):
#    for j in range(2):
#        ax8[i][j].set_ylabel('current[I]')
#        ax8[i][j].set_xlabel('RPM [-]')
#        ax8[i][j].set_ylim(0,22)
#        ax8[i][j].set_xlim(0,3500)
#        ax8[i][j].grid()
#        ax8[i][j].legend()
#
#ax8[0][0].set_title(f'RPM vs. current motor-pair 2-5')
#ax8[0][1].set_title(f'RPM vs. current motor-pair 1-6')
#ax8[1][0].set_title(f'RPM vs. current motor-pair 3-8')
#ax8[1][1].set_title(f'RPM vs. current motor-pair 4-7')
#
#fig8.tight_layout()